In [19]:
from pymongo import MongoClient
from passlib.context import CryptContext
import datetime
from pprint import pprint
from dateutil import parser

In [2]:
client = MongoClient(port=27017)
db = client.TransportDept
pwd_context = CryptContext(schemes=["sha512_crypt"], 
                           default="sha512_crypt", 
                           sha512_crypt__default_rounds=45000)

In [166]:
#Find bookings of next week
def retrive_next_week_booking(eid):
    booking_next_week=[]
    max_date = datetime.datetime.now()+datetime.timedelta(days=10)
    for bk in db.carhire.find({'Employee Eid':eid}):
        bk_date = parser.parse(bk['Date Time'].split('#')[0],dayfirst=True)
        if bk_date<=max_date:
            booking_next_week.append(bk['_id'])
    
    return booking_next_week

In [188]:
#Find details of current trip
def retrive_cur_trip_details(eid):
    details={}
    cur_time = datetime.datetime.now()
    for bk in db.carhire.find({'Employee Eid':eid}):
        bk_date = parser.parse(bk['Date Time'].split('#')[0],dayfirst=True)
        hr = bk['Date Time'].split('#')[1]
        #Get current booking
        if(bk_date.date()==cur_time.date()) and (hr==cur_time.hour()):
        #if True:
            details['Car Reg No'] = bk['Car Reg No']
            
            driver = db.employee.find_one({'eid':bk['Driver Eid']})
            
            if driver['MiddleName']=="":
                details['Driver Name']=driver['FirstName'] + " " + driver['LastName']
            else:
                details['Driver Name']=driver['FirstName'] + " " + driver['MiddleName'] + " " + driver['LastName']
            
            details['Driver Mobile'] = driver['mobile']
    
    return details
            

In [191]:
retrive_cur_trip_details(2)

{'Car Reg No': 9985715874,
 'Driver Mobile': '789687458',
 'Driver Name': 'Aastha Agarwal'}